In [1]:
import os
import getpass
from dotenv import load_dotenv
from tavily import TavilyClient
from langchain_tavily import TavilySearch
from langchain_tavily import TavilyExtract
from langchain_tavily import TavilyCrawl
from langchain_openai import ChatOpenAI
import datetime
from langgraph.prebuilt import create_react_agent
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.schema import HumanMessage
from IPython.display import Markdown
from langchain_community.utilities import SerpAPIWrapper
from langchain_community.agent_toolkits.load_tools import load_tools

# Load environment variables from .env file
load_dotenv()

# Prompt the user to securely input the API key if not already set in the environment
if not os.environ.get("TAVILY_API_KEY"):
    os.environ["TAVILY_API_KEY"] = getpass.getpass("TAVILY_API_KEY:\n")

if not os.environ.get("SERPAPI_API_KEY"):
    os.environ["SERPAPI_API_KEY"] = getpass.getpass("SERPAPI_API_KEY:\n")

if not os.environ.get("OPENAI_API_KEY"):
    os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter API key for OpenAI: ")

# Initialize the Tavily API client using the loaded or provided API key
tavily_client = TavilyClient(api_key=os.getenv("TAVILY_API_KEY"))
# Initialize the SerpApi tool with API key
serp_search = SerpAPIWrapper(serpapi_api_key=os.getenv("SERPAPI_API_KEY"))

In [2]:
# Define the LangChain search tool
search = TavilySearch(max_results=10, topic="general")

# Define the LangChain extract tool
extract = TavilyExtract(extract_depth="advanced")

# Define the LangChain crawl tool
crawl = TavilyCrawl()

serpapi_tool = load_tools(["serpapi"])

# gpt-4.0
gpt_4_0 = ChatOpenAI(model="gpt-4o-mini-2024-07-18", api_key=os.getenv("OPENAI_API_KEY"))

In [3]:
today = datetime.datetime.today().strftime("%A, %B %d, %Y")
tools = serpapi_tool + [search, extract, crawl]

# Create the web agent
web_agent = create_react_agent(
    model=gpt_4_0,
    tools=tools,
    prompt=ChatPromptTemplate.from_messages(
        [
            (
                "system",
                f"""
        You are a research agent equipped with advanced web tools: Tavily Web Search, Web Crawl, and Web Extract. Your mission is to conduct comprehensive, accurate, and up-to-date research, grounding your findings in credible web sources.

        **Today's Date:** {today}

        **Available Tools:**

        1. **Tavily Web Search**

        * **Purpose:** Retrieve relevant web pages based on a query.
        * **Usage:** Provide a search query to receive semantically ranked results, each containing the title, URL, and a content snippet.
        * **Best Practices:**

            * Use specific queries to narrow down results.
            * Optimize searches using parameters such as `search_depth`, `time_range`, `include_domains`, and `include_raw_content`.
            * Break down complex queries into specific, focused sub-queries.

        2. **SerpApi Web Search**

        * **Purpose:** Search Google and other engines for up-to-date information.
        * **Usage:** Provide a search query to retrieve results, including titles, snippets, and URLs.
        * **Best Practices:**

            * Use for broader or comparison queries when Tavily’s results are insufficient.
            * Try using both Tavily and SerpApi for comprehensive coverage.
            * Break down complex queries into specific, focused sub-queries.

        3. **Tavily Web Crawl**

        * **Purpose:** Explore a website's structure and gather content from linked pages for deep research and information discovery from a single source.
        * **Usage:** Input a base URL to crawl, specifying parameters such as `max_depth`, `max_breadth`, and `extract_depth`.
        * **Best Practices:**

            * Begin with shallow crawls and progressively increase depth.
            * Utilize `select_paths` or `exclude_paths` to focus the crawl.
            * Set `extract_depth` to "advanced" for comprehensive extraction.

        4. **Tavily Web Extract**

        * **Purpose:** Extract the full content from specific web pages.
        * **Usage:** Provide URLs to retrieve detailed content.
        * **Best Practices:**

            * Set `extract_depth` to "advanced" for detailed content, including tables and embedded media.
            * Enable `include_images` if image data is necessary.


        **Guidelines for Conducting Research:**

        * **Citations:** Always support findings with source URLs, clearly provided as in-text citations.
        * **Accuracy:** Rely solely on data obtained via provided tools—never fabricate information.
        * **Methodology:** Follow a structured approach:

        * **Thought:** Consider necessary information and next steps.
        * **Action:** Select and execute appropriate tools.
        * **Observation:** Analyze obtained results.
        * Repeat Thought/Action/Observation cycles as needed.
        * **Final Answer:** Synthesize and present findings with citations in markdown format.

        **Example Workflows:**

        **Workflow 1: Search Only**

        **Question:** What are recent news headlines about artificial intelligence?

        * **Thought:** I need quick, recent articles about AI.
        * **Action:** Use Tavily Web Search with the query "recent artificial intelligence news" and set `time_range` to "week".
        * **Observation:** Retrieved 10 relevant articles from reputable news sources.
        * **Final Answer:** Summarize key headlines with citations.

        **Workflow 2: Search and Extract**

        **Question:** Provide detailed insights into recent advancements in quantum computing.

        * **Thought:** I should find recent detailed articles first.
        * **Action:** Use Tavily Web Search with the query "recent advancements in quantum computing" and set `time_range` to "month".
        * **Observation:** Retrieved 10 relevant results.
        * **Thought:** I should extract content from the most comprehensive article.
        * **Action:** Use Tavily Web Extract on the most relevant URL from search results.
        * **Observation:** Extracted detailed information about quantum computing advancements.
        * **Final Answer:** Provide detailed insights summarized from extracted content with citations.

        **Workflow 3: Search and Crawl**

        **Question:** What are the latest advancements in renewable energy technologies?

        * **Thought:** I need recent articles about advancements in renewable energy.
        * **Action:** Use Tavily Web Search with the query "latest advancements in renewable energy technologies" and set `time_range` to "month".
        * **Observation:** Retrieved 10 articles discussing recent developments in solar panels, wind turbines, and energy storage.
        * **Thought:** To gain deeper insights, I'll crawl a relevant industry-leading renewable energy site.
        * **Action:** Use Tavily Web Crawl on the URL of a leading renewable energy industry website, setting `max_depth` to 2.
        * **Observation:** Gathered extensive content from multiple articles linked on the site, highlighting new technologies and innovations.
        * **Final Answer:** Provide a synthesized summary of findings with citations.

        ---

        You will now receive a research question from the user:

        """,
            ),
            MessagesPlaceholder(variable_name="messages"),
        ]
    ),
    name="web_agent",
)

In [4]:
# Test the web agent
inputs = {
    "messages": [
        HumanMessage(
            content="find all the fastest flights from Wellington to Ho Chi Minh City from today till November and their prices"
        )
    ]
}

# Stream the web agent's response
for s in web_agent.stream(inputs, stream_mode="values"):
    message = s["messages"][-1]
    if isinstance(message, tuple):
        print(message)
    else:
        message.pretty_print()

================================ Human Message =================================

find all the fastest flights from Wellington to Ho Chi Minh City from today till November and their prices
================================== Ai Message ==================================
Name: web_agent
Tool Calls:
  Search (call_WscwH9QxMj79hmCpuMbMVLjG)
 Call ID: call_WscwH9QxMj79hmCpuMbMVLjG
  Args:
    __arg1: fastest flights from Wellington to Ho Chi Minh City prices September to November 2025
================================= Tool Message =================================
Name: Search

["$241 is the best price for a one-way Wellington International Airport to Tan Son Nhat International Airport flight. · Don't wait until the last minute.", 'Search and book flights between Wellington and Ho Chi Minh City by comparing our fares shown on this page. ... Depart: 17 Nov 2025. From. $691*. Refreshed: 16 hrs ...', 'Cheap Flights from Wellington (WLG) to Ho Chi Minh City (SGN) start at $334 for one-way and $

In [5]:
Markdown(message.content)

Here are some of the fastest flights from Wellington (WLG) to Ho Chi Minh City (SGN) from September to November 2025, along with their prices:

1. **Flights on September 11, 2025**  
   - **Airline:** Qantas Airways  
   - **Price:** $325  
   - **Availability:** Only 3 seats left  
   - [View Details](https://us.trip.com/flights/wellington-to-ho-chi-minh-city/airfares-wlg-sgn/)

2. **Flights on October 8, 2025**  
   - **Airline:** Qantas Airways  
   - **Price:** $307  
   - **Availability:** Selling fast  
   - [View Details](https://us.trip.com/flights/wellington-to-ho-chi-minh-city/airfares-wlg-sgn/)

3. **Flights on November 2, 2025**  
   - **Airline:** Qantas Airways  
   - **Price:** $374  
   - **Availability:** Limited seats available  
   - [View Details](https://us.trip.com/flights/wellington-to-ho-chi-minh-city/airfares-wlg-sgn/)

4. **General October 2025 pricing**  
   - **Airline:** Air New Zealand  
   - **Price:** From $691 for one-way  
   - [View Details](https://www.airnewzealand.co.nz/flights/en-nz/flights-from-wellington-to-ho-chi-minh-city)

5. **General November 2025 pricing**  
   - **Airline:** Qantas Airways  
   - **Price:** From $691 for one-way  
   - [View Details](https://www.airnewzealand.co.nz/flights/en-nz/flights-from-wellington-to-ho-chi-minh-city)

Always check availability as prices and seats may vary. It's advised to book soon as some advertised seats are limited!